# Centroid Shift

## Import packages

In [1]:
import pandas as pd
import numpy as np
from astropy import constants
from astronavigation.deflection import *
from astronavigation.planets import Body, SolarSystem
from astronavigation.read_exo import *
import numpy.linalg as LA

np.set_printoptions(precision=54)

## Useful constants

In [2]:
# constants
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

# Create Solar System
ss = SolarSystem()

jupiter = ss.getPlanet('jupiter')

## Define problem

In [3]:
# observer
obs = 'earth'

# target exoplanets
targets_p = ['GU Psc b', 'HATS-43 b', 'CoRoT-10 b', 'TOI-132 b', 'HATS-52 b']

# target stars
targets_s = ['GU Psc', 'HATS-43', 'CoRoT-10', 'TOI-132', 'HATS-52']

# read exo catalogue
path = 'exo_archive.csv'
catalogue = pd.read_csv(path)

## Evaluate deflection and shift

In [7]:
# loop over targets
for i in range(len(targets_p)):
    
    # planet
    exo_p = getExo(targets_p[i], catalogue)
    a_p = getExoOrbRad(targets_p[i], catalogue)
    
    x_p = np.array([0, exo_p.dist, a_p])
       
    # star
    exo_s = getExoStar(targets_s[i], catalogue)
    
    x_s = np.array([0, 1, 0]) * (exo_s.dist)
    
    # obs
    x_obs = ss.getPlanet(obs).pos
    
    # impact angle grazing
    chi = a_p/(exo_p.dist + LA.norm(x_obs))
    #print(rad2muas(chi))
    
    # direction
    l0 = -np.array([np.sin(chi), np.cos(chi), 0])
    x_s = -LA.norm(x_s - x_obs) * l0 + x_obs
    
    dl = light_defl(l0, x_s, x_p, x_obs, eps, exo_p.mass)
    
    # centroid shift
    theta_E = einstein_ring(exo_p.mass, eps, x_p, x_s, x_obs)
    beta = chi / theta_E
    shift = cs_beta(beta, LA.norm(x_s - x_obs), LA.norm(x_s - x_p), theta_E)
    
    print(chi/theta_E)
    
    print(targets_p[i])
    print(f'defl: {rad2muas(LA.norm(dl))} muas')
    print(f'cs_shift: {rad2muas(shift)}')
    print(f'mass: {exo_p.mass/jupiter.mass} mJ')
    print(f'orb_rad: {a_p/AU} AU')
    print(f'dist: {exo_p.dist/pc} pc')
    print('\n')

1822054.8222799432
GU Psc b
defl: 0.0310650581517928 muas
cs_shift: 1.2669388094792312e-05
mass: 11.3 mJ
orb_rad: 2000.0000000000002 AU
dist: 47.5501 pc


nan
HATS-43 b
defl: 0.0 muas
cs_shift: nan
mass: 0.0 mJ
orb_rad: 0.0 AU
dist: 362.273 pc


26923.44457332029
CoRoT-10 b
defl: 142.2767316058531 muas
cs_shift: 4.301092867470476e-07
mass: 2.73 mJ
orb_rad: 0.1055 AU
dist: 338.3860000000001 pc


83183.92960614394
TOI-132 b
defl: 14.90446002784191 muas
cs_shift: 2.2956419908536498e-08
mass: 0.07048 mJ
orb_rad: 0.026 AU
dist: 163.67800000000003 pc


14462.984732346451
HATS-52 b
defl: 493.0367301168415 muas
cs_shift: 2.1511898559789796e-07
mass: 2.24 mJ
orb_rad: 0.02498 AU
dist: 555.134 pc




/var/folders/7k/393v523d5fx6hq7ntyqg05x00000gn/T/ipykernel_18770/1545454328.py:30: RuntimeWarning: invalid value encountered in double_scalars
  beta = chi / theta_E
/var/folders/7k/393v523d5fx6hq7ntyqg05x00000gn/T/ipykernel_18770/1545454328.py:33: RuntimeWarning: invalid value encountered in double_scalars
  print(chi/theta_E)


In [5]:
rad2muas(4*3.280*jupiter.mass*eps**2/(0.026*AU))

980.9326269721291